# Case Study 1: How Does a Bike-Share Navigate Speedy Success?

### Scenario
You are a junior data analyst working in the marketing analyst team at Cyclistic, a bike-share company in Chicago. The director
of marketing believes the company’s future success depends on maximizing the number of annual memberships. Therefore,
your team wants to understand how casual riders and annual members use Cyclistic bikes differently. From these insights,
your team will design a new marketing strategy to convert casual riders into annual members. But first, Cyclistic executives
must approve your recommendations, so they must be backed up with compelling data insights and professional data
visualizations.

### Business Task
The objective is to maximize the number of annual memberships by converting casual riders. 
Answering the following questions would give insights to possible ways that can be done.

1. How do annual members and casual riders use Cyclistic bikes differently?
2. Why would casual riders buy Cyclistic annual memberships?
3. How can Cyclistic use digital media to influence casual riders to become members?

### All about the Dataset
[Cyclistic bikes Dataset](https://divvy-tripdata.s3.amazonaws.com/index.html)  
The datasets have a different name because Cyclistic is a fictional company.  
The data is organised per month for every year period in zipped files.  
I will be using a 12 month period between 2021-10 to 2022-09 for this project.
The data is made available by Motivate International Inc. [License](https://www.divvybikes.com/data-license-agreement)

It contains 13 columns with the following names  
__ride_id__, rideable_type, __started_at__, ended_at,    	
__start_station_name__, start_station_id, __end_station_name__, end_station_id,  	
__start_lat__, start_lng, __end_lat__, end_lng, __member_casual__.

### Sort Filter in Excel using marcos(Cleaning Data)
* used macros to automate the cleaning process for the 12 excel files
* A new column named duration(mins) was created from the date time column 


   
 

### Import Libraries

In [34]:
#connection to PostgreSQL database
import psycopg2
import os
import pandas as pd


### Establish a connection to database 

In [31]:
conn = psycopg2.connect(database="cyclistic_bikes", user="postgres", password="Spily1988", host="127.0.0.1", port="5432")
print("Database connection established")

Database connection established


In [32]:
cur = conn.cursor()